In [98]:
import requests
import pandas as pd
import sqlalchemy as sa
from datetime import datetime, timedelta, timezone


In [99]:
# New API URL for historical weather data
api_url = 'https://archive-api.open-meteo.com/v1/archive'

# Database connection URL
connection_url = sa.engine.URL.create(
    drivername="mssql+pyodbc",
    username="wyx0108",
    password="2024!Schulich",
    host="mban2024-ms-sql-server.c1oick8a8ywa.ca-central-1.rds.amazonaws.com",
    port="1433",
    database="wyx0108_db",
    query={
        "driver": "ODBC Driver 18 for SQL Server",
        "TrustServerCertificate": "yes"
    }
)

# Create the database engine
engine = sa.create_engine(connection_url)

In [100]:
# Load all postal codes from the CSV
postal_codes_df = pd.read_csv('/Users/Owner/Desktop/ca_geo_dimension.csv')

# Filter for postal codes in Toronto
toronto_postal_codes_df = postal_codes_df[postal_codes_df['region'].str.contains("Toronto", case=False)]

# For now, select a subset of 15 postal codes
subset_size = 15
use_all_postal_codes = False  # Set this to True to process all postal codes

if use_all_postal_codes:
    selected_postal_codes = toronto_postal_codes_df
else:
    selected_postal_codes = toronto_postal_codes_df.sample(n=subset_size, random_state=1)

# Convert the selected postal codes to a list of dictionaries
postal_codes = selected_postal_codes.to_dict('records')


In [101]:
# Define the date range for historical data (e.g., last 7 days)
# Define the date range for historical data with a fixed end date
#end_date = "2024-07-30"  # Fixed end date
end_date = datetime.now(timezone.utc).strftime('%Y-%m-%d')  # Current date in UTC
start_date = (datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')  # 6 days before the end_date to cover 7 days

# Initialize list to store weather data
historical_weather_data = []

# Iterate over each postal code location
for location in postal_codes:
    # Define the parameters for the API call
    params = {
        'latitude': location['latitude'],  # Latitude
        'longitude': location['longitude'],  # Longitude
        'start_date': start_date,  # Start date in the format 'YYYY-MM-DD'
        'end_date': end_date,      # End date in the format 'YYYY-MM-DD'
        'daily': 'temperature_2m_mean,apparent_temperature_mean',  # Request daily mean temperature and apparent temperature data
        'timezone': 'UTC'  # Ensure timestamps are in UTC
    }
    
    # Make the API request
    response = requests.get(api_url, params=params)
    
    # Check if the API request was successful
    if response.status_code == 200:
        data = response.json()
        for i, timestamp in enumerate(data['daily']['time']):
            entry = {
                'city': location['region'],  # City or region
                'latitude': location['latitude'],  # Latitude
                'longitude': location['longitude'],  # Longitude
                'zipcode': location['zipcode'],  # Zipcode
                'temperature': data['daily']['temperature_2m_mean'][i],  # Mean daily temperature
                'feels_like': data['daily']['apparent_temperature_mean'][i],  # Mean daily apparent temperature (feels like)
                'timestamp_utc': timestamp  # Date in UTC
            }
            historical_weather_data.append(entry)
    else:
        print(f"Failed to fetch data for {location['zipcode']}: {response.status_code}")


In [102]:
# Convert the list of data to a DataFrame
df = pd.DataFrame(historical_weather_data)

# Convert timestamp_utc to a proper datetime object if not already
df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc']).dt.strftime('%Y-%m-%d %H:%M:%S')

# Print the DataFrame's column names and types
print(df.dtypes)



city              object
latitude         float64
longitude        float64
zipcode           object
temperature      float64
feels_like       float64
timestamp_utc     object
dtype: object


In [103]:
# Insert the cleaned data into the SQL database
df.to_sql(
    name='historical_weather',  # Specify the table name
    con=engine,  # Use the SQLAlchemy engine for database connection
    schema='uploads',  # Specify the schema name
    if_exists='replace',  # Replace the table if it already exists
    index=False,  # Do not write DataFrame index as a column
    dtype={
        'city': sa.types.VARCHAR(100),  # Define city column as VARCHAR(100)
        'latitude': sa.types.FLOAT,  # Define latitude column as FLOAT
        'longitude': sa.types.FLOAT,  # Define longitude column as FLOAT
        'zipcode': sa.types.VARCHAR(10),  # Define zipcode column as VARCHAR(10)
        'temperature': sa.types.DECIMAL(5, 2),  # Define temperature column as DECIMAL(5, 2)
        'feels_like': sa.types.DECIMAL(5, 2),  # Define feels_like column as DECIMAL(5, 2)
        'timestamp_utc': sa.types.DATETIME  # Define timestamp_utc column as DATETIME
    },
    method='multi'  # Enable multi-row insert for better performance
)

print("Historical weather data has been successfully stored.")




Historical weather data has been successfully stored.


In [104]:
df.head(10)

,city,latitude,longitude,zipcode,temperature,feels_like,timestamp_utc
0,Toronto,43.780775,-79.400830,M2N4N1,20.1,19.4,2024-07-26 00:00:00
1,Toronto,43.780775,-79.400830,M2N4N1,20.9,21.3,2024-07-27 00:00:00
2,Toronto,43.780775,-79.400830,M2N4N1,22.2,23.9,2024-07-28 00:00:00
3,Toronto,43.780775,-79.400830,M2N4N1,22.9,24.8,2024-07-29 00:00:00
4,Toronto,43.780775,-79.400830,M2N4N1,23.3,26.2,2024-07-30 00:00:00
5,Toronto,43.780775,-79.400830,M2N4N1,25.0,27.8,2024-07-31 00:00:00
6,Toronto,43.780775,-79.400830,M2N4N1,NaN,NaN,2024-08-01 00:00:00
7,Toronto,43.780775,-79.400830,M2N4N1,NaN,NaN,2024-08-02 00:00:00
8,Toronto,43.810684,-79.319894,M1W3H2,20.2,19.4,2024-07-26 00:00:00
9,Toronto,43.810684,-79.319894,M1W3H2,21.1,21.7,2024-07-27 00:00:00
